In [1]:
import re
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import nltk
import os

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\orkunmanap\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
ps = PorterStemmer()

file_info_spamicity = {}

paths = ['TEST\\ham', 'TEST\\spam']

In [4]:
import json

f = open('FILES\\spamicity.json',)
spamicity_possibilty = json.load(f)

default_spamicity = 0.5

In [5]:
for path in paths:
    for filename in os.listdir(path):
        try:
            input_program = open(path + "\\"+filename, "r").read()

            # Replace email address with 'emailaddress'
            input_program = re.sub(r'[\w\.-]+@[\w\.-]+\.\w+', ' emailadress ', input_program)

            # Replace urls with 'webaddress'
            input_program = re.sub(r'[\w\.-]+@[\w\.-]+\.\w+', ' webadress ', input_program)

            # Replace money symbol with 'money-symbol'
            input_program = re.sub(r'£|\$', 'money-symbol', input_program)

            # Replace 10 digit phone number with 'phone-number'
            input_program = re.sub(".*?(\(?\d{3}\D{0,3}\d{3}\D{0,3}\d{4}).*?", ' phone-number ', input_program)

            # Replace normal number with 'number'
            input_program = re.sub('\d', ' number ', input_program)

            # remove punctuation
            input_program = re.sub(r'[^\w\d\s]', ' ', input_program)

            # remove whitespace between terms with single space
            input_program = re.sub(r'\s+', ' ', input_program)

            # remove leading and trailing whitespace
            input_program = re.sub(r'^\s+|\s*?$', ' ', input_program)

            # change words to lower case
            input_program = input_program.lower()

            words = word_tokenize(input_program)

            spamicity_list = {}

            for w in words:
                lex = ps.stem(w)
                if lex not in spamicity_possibilty.keys():
                    #print(lex)
                    spamicity_list[lex] = default_spamicity
                else:
                    spamicity_list[lex] = spamicity_possibilty[lex]

            file_info_spamicity[filename] = spamicity_list
            
        except UnicodeDecodeError:
            continue

In [6]:
import heapq

In [7]:
file_info_highest_spam = {}
file_info_lowest_spam = {}

In [8]:
analyzed_lexeme = 8

In [9]:
threshold_value = 0.5

number_spam_as_ham = 0
number_spam_as_spam = 0
number_ham_as_ham = 0
number_ham_as_spam = 0

In [10]:
for path in paths:
    for filename in os.listdir(path):
        try:
            file_info_highest_spam[filename] = heapq.nlargest(analyzed_lexeme, file_info_spamicity[filename], key=file_info_spamicity           [filename].get)
            file_info_lowest_spam[filename] = heapq.nsmallest(analyzed_lexeme, file_info_spamicity[filename], key=file_info_spamicity           [filename].get)
        except KeyError:
            continue

In [11]:
file_spamicity = {}

In [12]:
for path in paths:
    for filename in os.listdir(path):
        try:
            spam_mult = 1
            spam_mult_minus1 = 1

            for i in file_info_highest_spam[filename]:
                spam_mult = spam_mult * float(file_info_spamicity[filename][i])
                spam_mult_minus1 = spam_mult_minus1 * float(1 - file_info_spamicity[filename][i])

            for j in file_info_lowest_spam[filename]:
                spam_mult = spam_mult * float(file_info_spamicity[filename][j])
                spam_mult_minus1 = spam_mult_minus1 * float(1 - file_info_spamicity[filename][j])

            #print(spam_mult)
            file_spamicity[filename] = spam_mult / (spam_mult + spam_mult_minus1)

            if file_spamicity[filename] < 0:
                print(filename)
                for i in file_info_highest_spam[filename]:
                    print(file_info_spamicity[filename][i], end=' ')
                print()
        except KeyError:
            continue

In [13]:
for file_val in file_spamicity:
    if file_spamicity[file_val] > threshold_value:
        if file_val[-11:] == 'ne_spam.txt':
            number_ham_as_spam = number_ham_as_spam + 1
        else:
            number_spam_as_spam = number_spam_as_spam + 1
    elif file_spamicity[file_val] < threshold_value:
        if file_val[-11:] == 'ne_spam.txt':
            number_ham_as_ham = number_ham_as_ham + 1
        else:
            number_spam_as_ham = number_spam_as_ham + 1

In [14]:
number_spam_as_ham

27

In [15]:
number_ham_as_spam

6

In [16]:
number_spam_as_spam

249

In [17]:
number_ham_as_ham

582

In [18]:
sum_files = number_spam_as_ham + number_spam_as_spam + number_ham_as_ham + number_ham_as_spam
correct_files = number_ham_as_ham + number_spam_as_spam 
accuracy = correct_files / sum_files

accuracy

0.9618055555555556

In [19]:
#with open("FILES\\file_spamicity.json", "w") as outfile: 
 #   json.dump(file_spamicity, outfile)